In [1]:
import numpy as np
from BPnumba.Individual import Ind
from BPmodule.BPGen import GetInstance
from BPnumba.AGH import createAG,GASearch
import matplotlib.pyplot as plt
from numba import types, njit,int64,jit,cuda,prange,vectorize
from numba.typed import List as NumbaList
from BPnumba.Poblation import CreateRotHPob,EvalPob
from BPnumba.BoxN import create_ItemBin,ItemBin,InstaceBoxes
from BPnumba.GeneticOperators import CrossOX,MutateInversion
import math

In [2]:
def zerovec(y, x): return np.zeros(shape=[x, y], dtype=np.float64)
createText = lambda x,y : np.savetxt(
    fname = "Results/"+x + ".csv",
    X= y,
    delimiter=" ",
    fmt='%g'
)

In [3]:
@njit#(parallel=True)
def BenchAG(i,boxes,bin,lst,lst2,lst3,maxItr,pop,rot):
    #fi = NumbaList(np.zeros(maxItr,dtype=np.float64))
    UBOXES = InstaceBoxes(boxes)
    pob = CreateRotHPob(pop =pop, ITEMS = UBOXES, rotType=rot)
    EvalPob(pob,UBOXES,bin,rot)
    
    ind:Ind=GASearch(maxItr=maxItr,ps=0.85,pc=0.75,pmr=0.05,mut=0,pob=pob.copy(),datos=UBOXES.copy(),contenedor=bin,rotType=rot)
    ind2:Ind=GASearch(maxItr=maxItr,ps=0.85,pc=0.75,pmr=0.05,mut=1,pob=pob.copy(),datos=UBOXES.copy(),contenedor=bin,rotType=rot)
    ind3:Ind=GASearch(maxItr=maxItr,ps=0.85,pc=0.75,pmr=0.05,mut=2,pob=pob.copy(),datos=UBOXES.copy(),contenedor=bin,rotType=rot)
    
    lst[i]=ind.fi
    lst2[i] = ind2.fi
    lst3[i] = ind3.fi
@njit(parallel=True)
def BAG(rango,contenedor,data:list[list[list[int]]],agL,agL2,agL3,maxItr,numpop,rotation):
    for p in prange(rango):
        BenchAG(i=p,boxes=data[p], bin=contenedor,maxItr= maxItr,lst= agL,lst2=agL2,lst3=agL3,pop=numpop,rot=rotation)

In [4]:
nm="P2A2"
BR1 = GetInstance(nm)
agL = zerovec(1, 100)
agL2 = zerovec(1, 100)
agL3 = zerovec(1, 100)

BAG(rango=1,contenedor=BR1[0],data=BR1[1],agL=agL,agL2=agL2,agL3=agL3,maxItr=500,numpop=50,rotation=6)

createText(nm+"/AG/"+"AG"+str(0),agL)
createText(nm+"/AG/"+"AG"+str(1),agL2)
createText(nm+"/AG/"+"AG"+str(2),agL3)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1mNameError: name 'lst1' is not defined[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function BenchAG at 0x000001552888C3A0>))[0m
[0m[1mDuring: typing of call at C:\Users\NicolasG\AppData\Local\Temp\ipykernel_10376\3450416049.py (18)
[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function BenchAG at 0x000001552888C3A0>))[0m
[0m[1mDuring: typing of call at C:\Users\NicolasG\AppData\Local\Temp\ipykernel_10376\3450416049.py (18)
[0m
[1m
File "C:\Users\NicolasG\AppData\Local\Temp\ipykernel_10376\3450416049.py", line 18:[0m
[1mdef BAG(rango,contenedor,data:list[list[list[int]]],agL,agL2,agL3,maxItr,numpop,rotation):
    <source elided>
    for p in prange(rango):
[1m        BenchAG(i=p,boxes=data[p], bin=contenedor,maxItr= maxItr,lst= agL,lst2=agL2,lst3=agL3,pop=numpop,rot=rotation)
[0m        [1m

In [ ]:
n= len(DataSet)
pop= 50
maxItr=1000
rot=6
pr_selec=0.85
pr_cruza = 0.75
pr_mutR=0.05
mutType=0
n,pop

In [ ]:
UBOXES = InstaceBoxes(DataSet)
Pob =   CreateRotHPob(pop =pop, ITEMS = UBOXES.copy(), rotType=rot)
EvalPob(Pob,UBOXES,contenedor,rot)

In [ ]:

fi = NumbaList(np.zeros(maxItr,dtype=np.float64))
ind2 = GASearch(maxItr=maxItr,ps=pr_selec,pc=pr_cruza,pmr=pr_mutR,mut=mutType,bestfi=fi,pob=Pob.copy(),datos=UBOXES.copy(),contenedor=contenedor,rotType=rot )

In [ ]:
ag = createAG(
    ps=pr_selec,
    pc=pr_cruza,
    pmr=pr_mutR,
    adaptive=True,
    mutType=1)
ind = ag.Train(maxItr,Pob.copy(),NumbaList(UBOXES.copy()),contenedor,rot)

In [ ]:
ag3 = createAG(
    ps=pr_selec,
    pc=pr_cruza,
    pmr=pr_mutR,
    adaptive=True,
    mutType=2)
ind3 = ag3.Train(maxItr,Pob.copy(),NumbaList(UBOXES.copy()),contenedor,rot)

In [ ]:
ind.fi,ind2.fi,ind3.fi

In [ ]:
n-ind.load,n-ind2.load,n-ind3.load

In [ ]:
plt.figure()
plt.plot(fi,color='red',label="MGAH")
plt.plot(ag.bestfi,color='blue',label="MGAH C1")
plt.plot(ag3.bestfi,color='green',label="MGAH C2")
plt.legend()
plt.ylim(0.5,1)
plt.show()